# Chronological frequency in OCRE

This notebook looks at the frequency of minting (measured in issues per annum) by emperor or issuing authority.  It uses version `3.0.0` of the `nomisma` library.



## Configure Jupyter notebook

First configure the Jupyter notebook. In addition to the `nomisma` library, we will use `plotly` for graph plots, and a `histoutils` package to simplify working with histograms.

In [1]:
// 1. Add maven repository where we can find our libraries
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

In [2]:
// 2. Make libraries available with `$ivy` imports:
import $ivy.`edu.holycross.shot::nomisma:3.0.0`
import $ivy.`edu.holycross.shot::histoutils:2.2.0`
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                                      

## Load the full OCRE data set

In [3]:
import edu.holycross.shot.nomisma._
val ocreCex = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/ocre-cite-ids.cex"
val ocre = OcreSource.fromUrl(ocreCex)

// Sanity check:
println(ocre.size + " records loaded.") 


Dec 27, 2019 5:02:32 PM wvlet.log.Logger log
INFO: Reading 50644 lines of CEX data.
2019-12-27 17:02:33.031Z  info [OcreSource] Created Ocre with 50644 issues.  - (OcreSource.scala:33)


50644 records loaded.


import edu.holycross.shot.nomisma._

ocreCex: String = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/ocre-cite-ids.cex"
ocre: Ocre = Ocre(
  Vector(
    OcreIssue(
      "3.com.43",
      "RIC III Commodus 43",
      "denarius",
      "ar",
      "commodus",
      "rome",
      "italy",
      "Head of Commodus, laureate, right",
      "M COMMODVS ANTONINVS AVG",
      "http://nomisma.org/id/commodus",
      "Roma, helmeted, draped, standing left, holding Victory in extended right hand and vertical spear in left hand",
      "TR P VII IMP V COS III P P",
      "http://collection.britishmuseum.org/id/person-institution/60208",
      Some(YearRange(182, Some(182)))
    ),
    OcreIssue(
      "9.thes.27B.iii",
      "RIC IX Thessalonica 27B: Subtype iii",
      "ae3",
      "ae",
      "valentinian_i",
      "thessalonica",
      "macedonia",
      "Bust of Valens, pearl-diademed, draped and cuirassed, right",
      "D N VALEN-S P F AVG",
      "http://nomisma.org/id/val

Set up environment for graph plotting with ploty.

In [4]:
// 1. Import plotly libraries, and set display defaults suggested for use in Jupyter NBs:
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)




import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


## Chronology

Overview of the whole data set:

In [5]:
// Use the date range object:
println("Total number of issues in OCRE: " + ocre.size)
println("Number of datable issues: " + ocre.datable.size)

println("Chronological range of issues in OCRE: " + ocre.dateRange)
println("Earliest issue: " + ocre.minDate)
println("Latest issues: " + ocre.maxDate)

Total number of issues in OCRE: 50644
Number of datable issues: 50613
Chronological range of issues in OCRE: 32 BCE:491 CE
Earliest issue: -32
Latest issues: 491


Cluster issues into distinct `Ocre` objects grouped by issuing authority, and sort these by their date range.

In [6]:
val byAuthority = ocre.issues.groupBy(_.authority).map{ 
    case (auth, issues) => auth -> Ocre(issues)
}
val byAuthorityChronological = byAuthority.toVector.sortBy{ _._2.dateRange.pointAverage}



byAuthority: Map[String, Ocre] = Map(
  "quietus" -> Ocre(
...
byAuthorityChronological: Vector[(String, Ocre)] = Vector(
  (
...

List authorities chronologically:

In [7]:
val authNames = byAuthorityChronological.map(_._1)
val authOcres =  byAuthorityChronological.map(_._2)
/*for ( (ocre, idx) <- authOcres.zipWithIndex) {
    println(authNames(idx) + ":  " + ocre.dateRange)
}*/

authNames: Vector[String] = Vector(
  "augustus",
...
authOcres: Vector[Ocre] = Vector(
  Ocre(
...

In [7]:
/*for (auth <- authNames) {
    println(auth + ": " + ocre.issueFrequency(auth) + " issues per year")
}*/

In [8]:
val avgFreq = ocre.size * 1.0f / ocre.dateSpan
println(ocre.size)
println(ocre.dateSpan)
println(avgFreq)

50644
524
96.64886


avgFreq: Float = 96.64886F

In [9]:
val augIssues = ocre.issuesForAuthority("augustus")
val augOcre = Ocre(augIssues)
println(augOcre.size)
println(augOcre.dateSpan)
println( augOcre.size * 1.0f / augOcre.dateSpan )
//println(ocre.issueFrequency("augustus"))

629
47
13.382978


augIssues: Vector[OcreIssue] = Vector(
  OcreIssue(
...
augOcre: Ocre = Ocre(
  Vector(
...

In [10]:
val summaries = for (auth <- authNames) yield {
    println(auth)
    val ocreForAuth =  Ocre(ocre.issuesForAuthority(auth))
    println(ocreForAuth.size)
    println(ocreForAuth.dateSpan)
    println(  ocreForAuth.size * 1.0f / ocreForAuth.dateSpan)
    (auth, ocreForAuth.size, ocreForAuth.dateSpan,  ocreForAuth.size * 1.0f / ocreForAuth.dateSpan)
}

augustus
629
47
13.382978
tiberius
95
24
3.9583333
gaius
63
5
12.6
claudius
126
14
9.0
nero
622
15
41.466667
clodius_macer
42
1
42.0
galba
522
2
261.0
otho
24
1
24.0
vitellius
177
1
177.0
vespasian
1588
11
144.36363
titus
543
3
181.0
domitian
859
16
53.6875
nerva
177
3
59.0
trajan
942
20
47.1
hadrian
2434
23
105.82609
antoninus_pius
2381
24
99.208336
marcus_aurelius
1819
20
90.95
anonymous
184
208
0.88461536
commodus
945
16
59.0625
pertinax
71
1
71.0
pescennius_niger
196
2
98.0
didius_julianus
52
1
52.0
clodius_albinus
154
5
30.8
septimius_severus
2170
21
103.333336
geta
39
2
19.5
caracalla
529
13
40.692307
diadumenian
37
2
18.5
macrinus
281
2
140.5
elagabalus
550
5
110.0
severus_alexander
942
14
67.28571
maximinus_thrax
168
4
42.0
gordian_i
16
1
16.0
pupienus
31
1
31.0
balbinus
31
1
31.0
gordian_ii
10
1
10.0
philip_the_arab
551
6
91.833336
jotapianus
5
2
2.5
pacatianus
12
2
6.0
mar_silbannacus
1
3
0.33333334
gordian_iii
552
22
25.09091
trajan_decius
531
3
177.0
hostilian
6
1
6.0
trebo

summaries: Vector[(String, Int, Int, Float)] = Vector(
  ("augustus", 629, 47, 13.382978F),
...

In [11]:
val regionPlot = Seq(
  Bar(x = authNames, y = summaries.map(_._4))
)
plot(regionPlot)

regionPlot: Seq[Bar] = List(
  Bar(
...
res10_1: String = "plot-5c4e31b3-2e3a-4b18-bfec-e93ee8fde366"